In [1]:
%pip install -qq -U embedchain

Note: you may need to restart the kernel to use updated packages.


In [2]:
from IPython.display import Markdown

In [3]:
from embedchain import App

pg_bot = App()
pg_bot.add("http://paulgraham.com/greatwork.html")

Doc content has not changed. Skipping creating chunks and embeddings


'161fcad59ba37a28e60a45f5355dab43'

In [4]:
res = pg_bot.query("how to do great work?", dry_run=True)
Markdown(res)


  Use the following pieces of context to answer the query at the end.
  If you don't know the answer, just say that you don't know, don't try to make up an answer.

  interesting, which you necessarily will if you're on the right path, the work will probably feel less burdensome than a lot of your peers'.The discoveries are out there, waiting to be made. Why not by you? Notes[1] I don't think you could give a precise definition of what counts as great work. Doing great work means doing something important so well that you expand people's ideas of what's possible. But there's no threshold for importance. It's a matter of degree, and often hard to judge at the

  Query: how to do great work?

  Helpful Answer:


In [5]:
res = pg_bot.retrieve_from_database("how to do great work?")
Markdown(res[0])

interesting, which you necessarily will if you're on the right path, the work will probably feel less burdensome than a lot of your peers'.The discoveries are out there, waiting to be made. Why not by you? Notes[1] I don't think you could give a precise definition of what counts as great work. Doing great work means doing something important so well that you expand people's ideas of what's possible. But there's no threshold for importance. It's a matter of degree, and often hard to judge at the

In [6]:
import fastrepl

# grab one from app.fastrepl.com
fastrepl.api_key = ""

In [7]:
generator = fastrepl.QuestionGenerator(source="http://paulgraham.com/greatwork.html")

In [8]:
ds = fastrepl.remote_runner(generator=generator).run()

Output()

Interrupted!
You can retrieve the job status with `run(job_id='fc-HkANDUul7t0KbSXOPn1jkr')`

In [9]:
ds = fastrepl.remote_runner(generator=generator).run(job_id="fc-HkANDUul7t0KbSXOPn1jkr")

Output()

In [10]:
ds

fastrepl.Dataset({
    features: ['question'],
    num_rows: 38
})

In [11]:
questions = ds["question"]
questions = questions[:10]
questions

['How can you nurture and protect your morale when working on ambitious projects?',
 'What are the advantages of having good colleagues when doing great work?',
 'Why is it important to avoid affectation in your work?',
 'How can curiosity be a driving force in doing great work?',
 'What are the three qualities that the work you choose needs to have?',
 'What are the drawbacks of getting paid for doing work close to what you want?',
 'Why is it difficult to figure out what to work on?',
 'Why is it important to have a more precise explanation for an idea that authorities discount?',
 'How can standup comedy be a source of new ideas?',
 'Why is it beneficial to seek out the best colleagues and work collaboratively on projects?']

In [12]:
answers = fastrepl.local_runner(fn=pg_bot.query).run(
    args_list=[(q,) for q in questions]
)
answers

Output()

['You can nurture and protect your morale when working on ambitious projects by maintaining a positive outlook on life. Being an optimist can help you stay motivated and focused on your goals. Additionally, it is important to take care of yourself physically and mentally by practicing self-care activities such as exercise, getting enough rest, and engaging in activities that bring you joy. Surrounding yourself with a supportive network of friends, family, or colleagues can also help boost your morale and provide encouragement during challenging times.',
 'The advantages of having good colleagues when doing great work include:\n\n1. Surprising insights: Good colleagues can offer unique perspectives and ideas that you may not have considered. Their different experiences and expertise can bring fresh insights to the table, leading to innovative solutions and better outcomes.\n\n2. Complementary skills: Good colleagues often possess different skills and strengths, which can complement your

In [13]:
contexts = fastrepl.local_runner(fn=pg_bot.retrieve_from_database).run(
    args_list=[(q,) for q in questions]
)
contexts

Output()

[["have to become a manager, and managing well takes aptitude and interest like any other kind of work. If you don't have them, there is no middle path: you must either force yourself to learn management as a second language, or avoid such projects. [27] Husband your morale. It's the basis of everything when you're working on ambitious projects. You have to nurture and protect it like a living organism.Morale starts with your view of life. You're more likely to do great work if you're an optimist,"],
 ['the fakeness is what shows in the work. [12]The temptation to be someone else is greatest for the young. They often feel like nobodies. But you never need to worry about that problem, because it\'s self-solving if you work on sufficiently ambitious projects. If you succeed at an ambitious project, you\'re not a nobody; you\'re the person who did it. So just do the work and your identity will take care of itself. "Avoid affectation" is a useful rule so far as it goes, but how would you e

In [14]:
ds = fastrepl.Dataset.from_dict(
    {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
    }
)
ds

fastrepl.Dataset({
    features: ['question', 'answer', 'contexts'],
    num_rows: 10
})

In [15]:
evaluator = fastrepl.RAGEvaluator(node=fastrepl.RAGAS(metric="AnswerRelevancy"))

In [16]:
result = fastrepl.local_runner(evaluator=evaluator, dataset=ds).run(show_progress=False)
result

fastrepl.Dataset({
    features: ['question', 'answer', 'contexts', 'result'],
    num_rows: 10
})

In [17]:
result["result"]

[0.9905149669187113,
 1.0000000000000002,
 1.0,
 0.8224544027340016,
 0.7210508405457526,
 0.7865917085330983,
 0.8919254408115931,
 0.7868346213471876,
 0.7260811619842708,
 0.9852220816665399]